In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class CF(object):
    """docstring for CF"""
    def __init__(self, Y_data, k, dist_func = cosine_similarity, uuCF = 1):
        self.uuCF = uuCF # user-user (1) or item-item (0) CF
        self.Y_data = Y_data if uuCF else Y_data[:, [1, 0, 2]]
        self.k = k
        self.dist_func = dist_func
        self.Ybar_data = None 
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(self.Y_data[:, 0])) + 1 
        self.n_items = int(np.max(self.Y_data[:, 1])) + 1
    
    def add(self, new_data):
        """
        Update Y_data matrix when new ratings come.
        For simplicity, suppose that there is no new user or item.
        """
        self.Y_data = np.concatenate((self.Y_data, new_data), axis = 0)
    
    def normalize_Y(self):
        users = self.Y_data[:, 0] # all users - first col of the Y_data
        self.Ybar_data = self.Y_data.copy()
        self.mu = np.zeros((self.n_users,))
        for n in range(self.n_users):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data[ids, 1] 
            # and the corresponding ratings 
            ratings = self.Y_data[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        ################################################
        # form the rating matrix as a sparse matrix. Sparsity is important 
        # for both memory and computing efficiency. For example, if #user = 1M, 
        # #item = 100k, then shape of the rating matrix would be (100k, 1M), 
        # you may not have enough memory to store this. Then, instead, we store 
        # nonzeros only, and, of course, their locations.
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:, 2],
            (self.Ybar_data[:, 1], self.Ybar_data[:, 0])), (self.n_items, self.n_users))
        self.Ybar = self.Ybar.tocsr()

    def similarity(self):
        eps = 1e-6
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    
        
    def refresh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity() 
        
    def fit(self):
        self.refresh()
        
    
    def __pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        # Step 1: find all users who rated i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # Step 2: 
        users_rated_i = (self.Y_data[ids, 0]).astype(np.int32)
        # Step 3: find similarity btw the current user and others 
        # who already rated i
        sim = self.S[u, users_rated_i]
        # Step 4: find the k most similarity users
        a = np.argsort(sim)[-self.k:] 
        # and the corresponding similarity levels
        nearest_s = sim[a]
        # How did each of 'near' users rated item i
        r = self.Ybar[i, users_rated_i[a]]
        if normalized:
            # add a small number, for instance, 1e-8, to avoid dividing by 0
            return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8)

        return (r*nearest_s)[0]/(np.abs(nearest_s).sum() + 1e-8) + self.mu[u]
    
    def pred(self, u, i, normalized = 1):
        """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        if self.uuCF: return self.__pred(u, i, normalized)
        return self.__pred(i, u, normalized)
            
    
    def recommend(self, u):
        """
        Determine all items should be recommended for user u.
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    recommended_items.append(i)
        
        return recommended_items 
    
    def recommend2(self, u):
        """
        Determine all items should be recommended for user u.
        The decision is made based on all i such that:
        self.pred(u, i) > 0. Suppose we are considering items which 
        have not been rated by u yet. 
        """
        ids = np.where(self.Y_data[:, 0] == u)[0]
        items_rated_by_u = self.Y_data[ids, 1].tolist()              
        recommended_items = []
    
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                rating = self.__pred(u, i)
                if rating > 0: 
                    recommended_items.append(i)
        
        return recommended_items 

    def print_recommendation(self):
        """
        print all items which should be recommended for each user 
        """
        print ('Recommendation: ')
        for u in range(self.n_users):
            recommended_items = self.recommend(u)
            if self.uuCF:
                print ('    Recommend item(s):', recommended_items, 'for user', u)
            else: 
                print ('    Recommend item', u, 'for user(s) : ', recommended_items)

In [2]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.values
rate_test = ratings_test.values

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [3]:
rs = CF(rate_train, k = 30, uuCF = 0)
rs.fit()

/home/yntn/.local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/yntn/.local/lib/python3.5/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [4]:
rs.print_recommendation()

Recommendation: 
    Recommend item 0 for user(s) :  [3, 6, 7, 8, 10, 11, 13, 18, 21, 23, 27, 29, 33, 34, 38, 44, 45, 46, 47, 51, 52, 59, 68, 79, 84, 85, 86, 89, 90, 92, 95, 102, 104, 109, 111, 112, 113, 116, 117, 118, 122, 125, 126, 133, 135, 138, 139, 141, 142, 151, 153, 156, 157, 158, 163, 164, 165, 168, 169, 172, 174, 179, 184, 185, 186, 187, 188, 189, 194, 195, 196, 201, 206, 207, 211, 213, 214, 217, 219, 222, 223, 224, 232, 238, 239, 240, 251, 256, 257, 258, 260, 263, 266, 269, 271, 272, 281, 282, 283, 284, 299, 303, 305, 308, 309, 314, 316, 317, 320, 321, 322, 327, 332, 333, 334, 336, 340, 345, 350, 351, 354, 358, 361, 363, 365, 366, 367, 368, 369, 371, 372, 374, 375, 376, 381, 382, 383, 389, 391, 395, 396, 399, 407, 413, 414, 417, 419, 420, 422, 425, 426, 427, 432, 435, 438, 439, 442, 450, 452, 461, 463, 468, 472, 473, 474, 475, 476, 480, 488, 498, 500, 501, 503, 505, 506, 509, 510, 512, 513, 515, 518, 521, 526, 527, 528, 529, 537, 538, 540, 545, 548, 550, 554, 555, 556, 557, 5

    Recommend item 6 for user(s) :  [1, 3, 4, 7, 10, 11, 17, 18, 21, 28, 29, 30, 33, 34, 35, 38, 40, 41, 45, 50, 60, 66, 69, 70, 72, 74, 76, 78, 79, 80, 82, 83, 85, 89, 90, 92, 96, 97, 102, 104, 111, 113, 120, 122, 124, 126, 127, 130, 131, 132, 133, 135, 136, 137, 138, 139, 142, 151, 152, 153, 156, 163, 164, 165, 167, 168, 172, 173, 178, 179, 181, 184, 186, 187, 194, 202, 204, 209, 211, 214, 217, 224, 227, 230, 231, 232, 238, 239, 245, 246, 252, 253, 256, 257, 258, 259, 260, 262, 265, 271, 272, 273, 281, 282, 283, 286, 289, 290, 297, 303, 306, 308, 309, 310, 311, 312, 314, 316, 317, 318, 321, 323, 329, 335, 336, 337, 344, 349, 350, 355, 357, 358, 364, 365, 368, 369, 370, 372, 374, 375, 376, 380, 382, 383, 387, 388, 389, 390, 391, 394, 395, 397, 399, 408, 410, 413, 414, 417, 418, 419, 420, 422, 425, 426, 427, 432, 435, 437, 439, 440, 442, 443, 450, 452, 455, 461, 463, 468, 472, 474, 475, 476, 480, 484, 486, 488, 490, 493, 494, 501, 502, 503, 505, 506, 507, 511, 512, 515, 517, 518, 519, 

    Recommend item 10 for user(s) :  [1, 3, 8, 11, 13, 17, 18, 20, 21, 22, 24, 28, 29, 30, 31, 33, 34, 35, 37, 40, 41, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 59, 64, 65, 66, 68, 69, 70, 71, 73, 76, 77, 78, 79, 82, 83, 85, 86, 87, 88, 90, 92, 94, 95, 96, 97, 105, 107, 111, 112, 113, 117, 122, 124, 125, 127, 130, 132, 133, 135, 136, 137, 138, 140, 142, 145, 146, 147, 149, 150, 151, 152, 153, 154, 156, 158, 163, 164, 165, 167, 168, 172, 175, 178, 179, 181, 184, 185, 186, 188, 194, 195, 198, 202, 204, 207, 209, 214, 217, 218, 219, 220, 224, 227, 231, 232, 238, 239, 240, 241, 242, 243, 250, 251, 252, 256, 258, 259, 260, 262, 263, 264, 266, 269, 273, 277, 280, 281, 282, 283, 284, 288, 289, 293, 297, 301, 303, 305, 306, 308, 309, 310, 311, 314, 316, 317, 318, 319, 321, 323, 332, 334, 335, 336, 337, 339, 347, 349, 350, 353, 354, 355, 356, 358, 364, 365, 366, 368, 369, 370, 371, 372, 374, 375, 376, 378, 380, 381, 382, 383, 385, 387, 393, 394, 395, 396, 397, 399, 401, 402, 406, 411, 414, 417, 4

    Recommend item 15 for user(s) :  [1, 3, 6, 7, 8, 10, 11, 13, 15, 17, 21, 22, 23, 24, 30, 33, 35, 36, 37, 43, 44, 45, 46, 47, 51, 55, 58, 59, 61, 63, 65, 66, 68, 72, 75, 77, 78, 83, 87, 88, 89, 92, 93, 95, 96, 97, 98, 102, 105, 107, 111, 112, 114, 116, 117, 118, 121, 122, 125, 129, 131, 133, 134, 135, 136, 137, 138, 142, 145, 146, 150, 151, 153, 156, 157, 158, 161, 163, 164, 165, 167, 168, 169, 172, 173, 175, 179, 183, 184, 186, 187, 189, 199, 209, 210, 211, 212, 213, 214, 215, 219, 220, 225, 229, 231, 232, 234, 238, 239, 240, 241, 243, 244, 246, 248, 249, 251, 255, 256, 257, 258, 259, 260, 262, 263, 266, 269, 271, 273, 277, 280, 281, 282, 283, 284, 286, 290, 291, 294, 295, 297, 299, 300, 305, 308, 309, 311, 314, 316, 318, 319, 321, 323, 329, 330, 331, 333, 336, 338, 339, 340, 342, 344, 347, 349, 350, 352, 353, 354, 356, 358, 359, 363, 364, 365, 366, 370, 371, 372, 375, 378, 382, 383, 384, 387, 390, 391, 392, 393, 402, 403, 405, 410, 411, 413, 414, 415, 418, 419, 423, 426, 427, 432,

    Recommend item 20 for user(s) :  [1, 3, 6, 8, 9, 10, 11, 13, 15, 17, 18, 22, 23, 24, 26, 28, 29, 30, 33, 34, 35, 37, 40, 41, 42, 44, 45, 47, 49, 51, 53, 54, 55, 56, 64, 65, 66, 68, 69, 70, 77, 78, 82, 83, 84, 85, 87, 88, 89, 90, 92, 93, 95, 96, 98, 102, 105, 106, 107, 111, 112, 116, 117, 118, 119, 120, 121, 122, 123, 125, 127, 129, 131, 133, 134, 135, 136, 137, 138, 140, 141, 142, 147, 149, 150, 152, 153, 155, 156, 157, 158, 161, 163, 164, 165, 167, 168, 169, 171, 172, 176, 177, 181, 183, 184, 185, 186, 187, 189, 191, 192, 194, 199, 202, 207, 209, 211, 212, 213, 214, 215, 219, 220, 224, 225, 229, 230, 232, 234, 238, 239, 241, 242, 243, 248, 249, 250, 251, 252, 256, 257, 258, 259, 260, 262, 263, 266, 269, 270, 271, 273, 279, 280, 283, 284, 285, 286, 287, 289, 291, 294, 295, 297, 299, 306, 308, 309, 311, 313, 314, 316, 317, 318, 319, 320, 323, 331, 332, 337, 338, 339, 340, 342, 343, 346, 347, 349, 350, 351, 354, 356, 357, 358, 359, 361, 363, 365, 366, 367, 368, 369, 370, 371, 372, 37

...

    Recommend item 107 for user(s) :  [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 22, 23, 24, 27, 28, 29, 30, 31, 33, 35, 37, 38, 39, 40, 41, 44, 45, 46, 47, 49, 51, 52, 53, 54, 55, 57, 58, 59, 61, 63, 64, 65, 66, 68, 70, 71, 72, 74, 75, 76, 77, 78, 79, 80, 83, 86, 88, 89, 92, 93, 94, 95, 96, 97, 98, 104, 106, 107, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 125, 127, 128, 129, 131, 132, 135, 136, 137, 138, 139, 141, 142, 144, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 159, 161, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 175, 176, 177, 179, 181, 182, 183, 184, 185, 186, 187, 188, 191, 196, 198, 199, 200, 201, 202, 203, 204, 209, 211, 212, 213, 214, 217, 219, 224, 225, 226, 229, 231, 232, 234, 238, 240, 241, 242, 243, 245, 247, 249, 250, 251, 252, 255, 256, 258, 259, 261, 262, 263, 269, 271, 272, 273, 277, 279, 281, 282, 283, 284, 285, 290, 291, 293, 294, 295, 297, 299, 300, 302, 303, 304, 305, 306, 307, 309, 310, 311, 312, 314, 3

    Recommend item 111 for user(s) :  [3, 6, 8, 9, 10, 11, 13, 15, 17, 23, 24, 26, 29, 30, 33, 34, 35, 38, 42, 44, 45, 46, 47, 51, 53, 55, 59, 60, 64, 66, 68, 69, 70, 78, 83, 86, 87, 88, 89, 92, 94, 95, 96, 97, 102, 105, 107, 111, 112, 114, 116, 117, 118, 119, 123, 125, 126, 129, 131, 133, 135, 136, 137, 138, 142, 150, 151, 153, 156, 158, 159, 163, 165, 169, 171, 172, 177, 178, 181, 184, 185, 186, 187, 188, 194, 202, 203, 209, 210, 211, 212, 213, 219, 220, 224, 225, 229, 230, 232, 238, 239, 240, 241, 242, 243, 248, 249, 250, 251, 252, 255, 256, 258, 259, 260, 262, 263, 266, 269, 271, 273, 275, 277, 280, 282, 283, 284, 285, 287, 290, 291, 293, 294, 297, 299, 300, 306, 307, 308, 309, 311, 313, 314, 316, 317, 318, 319, 320, 323, 329, 330, 331, 332, 336, 338, 339, 340, 342, 343, 347, 350, 351, 353, 354, 356, 357, 358, 359, 363, 366, 368, 369, 370, 371, 372, 376, 378, 380, 382, 383, 387, 390, 392, 393, 395, 397, 398, 399, 401, 402, 403, 405, 407, 411, 415, 418, 419, 422, 423, 426, 427, 431,

    Recommend item 115 for user(s) :  [1, 3, 6, 7, 8, 10, 11, 15, 18, 21, 23, 26, 27, 28, 29, 30, 33, 35, 36, 37, 38, 40, 43, 44, 45, 46, 50, 53, 55, 56, 59, 63, 64, 66, 67, 68, 69, 70, 72, 75, 77, 79, 82, 83, 85, 86, 87, 88, 89, 90, 92, 93, 94, 95, 96, 102, 105, 106, 107, 111, 113, 114, 116, 117, 118, 120, 121, 122, 123, 125, 127, 129, 131, 132, 136, 137, 138, 139, 145, 146, 148, 150, 151, 153, 156, 158, 159, 163, 164, 166, 168, 169, 171, 172, 173, 175, 176, 177, 179, 181, 187, 188, 194, 199, 202, 203, 204, 209, 210, 212, 213, 215, 217, 218, 219, 220, 224, 225, 226, 230, 231, 232, 234, 240, 241, 243, 244, 245, 246, 248, 250, 251, 252, 255, 256, 257, 258, 259, 262, 266, 269, 271, 272, 273, 277, 279, 281, 282, 283, 286, 287, 290, 291, 293, 294, 295, 297, 299, 305, 306, 310, 311, 312, 313, 314, 315, 317, 318, 319, 321, 322, 323, 329, 330, 331, 336, 337, 338, 340, 341, 343, 344, 347, 348, 349, 350, 352, 354, 355, 356, 357, 358, 360, 363, 364, 366, 367, 368, 370, 371, 372, 374, 375, 377, 3

    Recommend item 120 for user(s) :  [3, 7, 8, 11, 15, 17, 23, 27, 34, 37, 45, 47, 53, 64, 78, 85, 87, 89, 90, 92, 95, 96, 97, 104, 105, 109, 112, 116, 117, 121, 124, 126, 130, 132, 133, 138, 139, 142, 164, 165, 168, 169, 171, 172, 173, 178, 179, 184, 195, 196, 211, 219, 223, 224, 239, 241, 244, 246, 249, 257, 259, 260, 262, 263, 264, 271, 273, 280, 281, 282, 291, 299, 303, 308, 309, 316, 321, 323, 332, 336, 339, 344, 349, 350, 354, 355, 358, 361, 365, 366, 368, 369, 370, 371, 372, 374, 376, 383, 389, 393, 397, 399, 401, 407, 411, 414, 420, 426, 427, 435, 439, 450, 461, 463, 468, 469, 475, 476, 480, 486, 488, 490, 501, 505, 509, 510, 511, 513, 514, 522, 526, 527, 528, 529, 538, 548, 549, 556, 557, 562, 573, 575, 584, 588, 595, 596, 597, 598, 599, 601, 610, 611, 612, 615, 619, 627, 628, 630, 631, 639, 640, 648, 657, 675, 676, 677, 684, 685, 686, 687, 688, 690, 693, 700, 704, 711, 719, 720, 731, 739, 751, 753, 754, 760, 766, 769, 771, 774, 776, 779, 782, 783, 784, 790, 800, 801, 807, 80

KeyboardInterrupt: 

In [5]:
n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1], normalized = 0)
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print ('User-user CF, RMSE =', RMSE)

User-user CF, RMSE = 0.9867912132705384


In [6]:
s = CF(rate_train, k = 30, uuCF = 0)
rs.fit()

n_tests = rate_test.shape[0]
SE = 0 # squared error
for n in range(n_tests):
    pred = rs.pred(rate_test[n, 0], rate_test[n, 1], normalized = 0)
    SE += (pred - rate_test[n, 2])**2 

RMSE = np.sqrt(SE/n_tests)
print ('Item-item CF, RMSE =', RMSE)

Item-item CF, RMSE = 0.9867912132705384
